# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
import os
import time

# Set working dir the same as the `.env` file
os.chdir('../../01_materials/labs/')
from dotenv import load_dotenv
load_dotenv() 

%reload_ext dotenv
%dotenv


In [2]:

import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
# Load the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')
print(f"PRICE_DATA directory: {price_data_dir}")

parquet_files = glob(os.path.join(price_data_dir, '*/*/*.parquet'))
print(f"Parquet files found: {parquet_files[:5]}")  # Print the first few files
dd_px = dd.read_parquet(parquet_files,npartitions=len("ticker")).set_index("ticker")


PRICE_DATA directory: ../../05_src/data/prices/
Parquet files found: ['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
# Write your code below.

import numpy as np



dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close = lambda x: x['Close']
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1)
).assign(    
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))


C:\Users\skiev\AppData\Local\Temp\ipykernel_20452\2736475020.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [10]:
# Write your code below.
# Convert to pandas DataFrame
dd_px = dd_px.repartition(npartitions=5)
pd_feat = dd_feat.compute()

# Add rolling average return calculation with a window of 10 days
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(window=10).mean()

# Display the result
print(pd_feat.tail())



             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
GILD   2024-06-24  71.000000  72.070000  70.559998  70.730003  70.730003   
GILD   2024-06-25  70.639999  71.480003  69.529999  69.610001  69.610001   
GILD   2024-06-26  69.190002  70.230003  68.900002  69.309998  69.309998   
GILD   2024-06-27  69.139999  69.589996  68.519997  68.830002  68.830002   
GILD   2024-06-28  68.830002  69.000000  68.059998  68.610001  68.610001   

          Volume       sector      subsector  year  Close_lag_1  Adj_Close  \
ticker                                                                       
GILD    10915500  Health Care  Biotechnology  2024    70.669998  70.730003   
GILD     9401500  Health Care  Biotechnology  2024    70.730003  69.610001   
GILD     5795200  Health Care  Biotechnology  2024    69.610001  69.309998   
GILD     7176300  Health Care  Biotechnology  2024    69.309998  68.830002   

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

    + It is not strictly necessary to convert a Dask DataFrame to a pandas DataFrame to calculate the moving average return. Dask DataFrames are designed to handle large datasets that do not fit into memory and provide functionality similar to pandas, including operations like moving averages.
    
+ Would it have been better to do it in Dask? Why?
    + Yes, it would generally be better to perform the moving average calculation in Dask, especially if you are working with large datasets that do not fit into memory. Here are some reasons why:
        + Dask is designed to handle large datasets by breaking them into smaller, manageable partitions and performing computations in parallel. This allows you to work with data that exceeds the available memory on a single machine.
        + Dask can distribute computations across multiple cores or even multiple machines, which can significantly speed up processing times for large datasets.
        + Dask processes data in chunks, meaning it only loads a portion of the data into memory at any given time. This prevents memory overflow issues that can occur when trying to process large datasets with pandas.
        + Dask's API is very similar to pandas, making it easy to transition code from pandas to Dask with minimal changes.
        + Converting between Dask and pandas can be time-consuming and memory-intensive. By staying within Dask, you avoid the overhead associated with these conversions.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.